# Pydantic Basics

- Pydantic is a data validation and settings management using Python type annotations.
- Pydantic enforces type hints at runtime, and provides user friendly errors when data is invalid.
- Using Pydantic models to define the schema of the data, and then validate the data against it.

Benefits of Pydantic:
- IDE autocompletion and type hinting.
- Data validation
- JSON serialization

In [1]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str
    email: str
    password: str

### Creating Instances of Pydantic Models
- Get auto-completion with the model's attributes.

In [2]:
# Initialize the User class with input values
user = User(id=1, name="John Doe", email="hi@gmail.com", password="secret")

# user object is now a dictionary
user_dict = {"id": 1, "name": "John Doe", "email": "john@gmail.com", "password": "123456"}

# Create a User object from a dictionary using the ** operator to unpack the dictionary
user = User(**user_dict)

### Accessing Attributes of Pydantic Models
- Access the attributes of the model using the dot notation.

In [3]:
print(user.id)
print(user.name)
print(user.email)
print(user.password)

1
John Doe
john@gmail.com
123456


## Type Annotations and Default Values
- Pydantic uses Python type annotations to define the schema of the data.

### Data Validation

In [4]:
# Default values
class User(BaseModel):
    id: int
    name: str
    email: str
    password: str
    # Default value for bio
    bio: str = "No bio available"


user = User(
    id=1,
    name="John Doe",
    email="john@gmail.com",
    password="123456",
    bio="A Python developer",
)

print(user)

id=1 name='John Doe' email='john@gmail.com' password='123456' bio='A Python developer'


### Type Annotations
- Pydantic uses Python type annotations to define the schema of the data.
- Pydantic enforces type hints at runtime, and provides user friendly errors when data is invalid.
- Pydantic models can be used to validate data against the schema.

In [5]:
from typing import Union, Optional, List


class User(BaseModel):
    id: int
    name: str
    email: str
    password: str
    bio: str = "No bio available"
    # list of strings mandatory
    skills: List[str] = []

    # Optional is used to specify that the age field is not required
    age: Optional[int] = None

    # Union is used to specify that the middle_name can be either a string or None
    middle_name: Union[str, None] = None

    # list of strings optional, default value is None
    hobbies: Optional[List[str]] = None

    # list of union of strings and integers, default value is an empty list. can be either a string or an integer
    friends: List[Union[str, int]] = []


user = User(
    id=1,
    name="John Doe",
    email="john@gmail.com",
    password="123456",
    bio="A Python developer",
    skills=["Python", "Django"],
    age=25,
    middle_name=None,
    hobbies=["Reading", "Gaming"],
    friends=[1, "Jane Doe"],
)

print(user)

id=1 name='John Doe' email='john@gmail.com' password='123456' bio='A Python developer' skills=['Python', 'Django'] age=25 middle_name=None hobbies=['Reading', 'Gaming'] friends=[1, 'Jane Doe']


## Nested Models
- Pydantic models can be nested inside other models.
- Nested models can be used to define the schema of the data.

In [6]:
class Post(BaseModel):
    title: str
    content: str
    user: User


post = Post(
    title="Hello, World!",
    content="This is the first post.",
    user=user,
)

print(post)

title='Hello, World!' content='This is the first post.' user=User(id=1, name='John Doe', email='john@gmail.com', password='123456', bio='A Python developer', skills=['Python', 'Django'], age=25, middle_name=None, hobbies=['Reading', 'Gaming'], friends=[1, 'Jane Doe'])


## Data Validation

In [7]:
# Type validation is performed when the object is created
user = User(id=1, name="John Doe", email="john@gmail.com", password="123456")

### Built-in Validators
- Pydantic provides built-in validators for common data types.

In [8]:
from pydantic import EmailStr


class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    password: str


# EmailStr is a Pydantic data type that validates email addresses
user = User(id=1, name="John Doe", email="john@gmail.com", password="123456")
print(user)

id=1 name='John Doe' email='john@gmail.com' password='123456'


In [9]:
# user = User(id=1, name="John Doe", email="johngmail.com", password="123456")
print(user)

id=1 name='John Doe' email='john@gmail.com' password='123456'


### Field Validators

- BeforeValidator: Validate the data before inner validators are applied.
- AfterValidator: Validate the data after inner validators are applied.
- Wrapvalidator: Wrap the data in a custom class before it is assigned to the field.

In [5]:
from typing import Annotated
from pydantic import BaseModel, BeforeValidator, AfterValidator, WrapValidator, EmailStr


# Define the validation functions
def clean_name(value: str) -> str:
    print(f"Cleaning name: {value}")
    return value.strip()


def hash_password(value: str) -> int:
    print(f"Hashing password: {value}")
    return hash(value)


def log_email(value: str, handler) -> str:
    print(f"Validating email: {value}")
    return handler(value)


class User(BaseModel):
    id: int
    # Annotated is used to apply validators to fields in Pydantic models
    # Apply BeforeValidator using Annotated
    name: Annotated[str, BeforeValidator(clean_name)]

    # Apply WrapValidator using Annotated
    email: Annotated[EmailStr, WrapValidator(log_email)]

    # Apply AfterValidator using Annotated
    password: Annotated[str, AfterValidator(hash_password)]


user = User(id=1, name="  John Doe  ", email="john@example.com", password="secret123")

print(user)

Cleaning name:   John Doe  
Validating email: john@example.com
Hashing password: secret123
id=1 name='John Doe' email='john@example.com' password=2398555695433270205


In [8]:
user = User(id=1, name=" John Doe ", email="john@gmail.com", password="123456")
print(user)

Cleaning name:  John Doe 
Validating email: john@gmail.com
Hashing password: 123456
id=1 name='John Doe' email='john@gmail.com' password=5268750335886593586
